### IMDB 영화 리뷰 데이터 분석

- 데이터 : https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [17]:
import pandas as pd 

df = pd.read_csv('IMDB Dataset.csv')
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [2]:
df.info() # 결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [3]:
# 공백 및 영어 대소문자를 제외한 모든 문자 제거
import re 
df['review'] = [re.sub('[^A-Za-z\s]', '', s).lower() for s in df['review']]
df['review'].head()

0    one of the other reviewers has mentioned that ...
1    a wonderful little production br br the filmin...
2    i thought this was a wonderful way to spend ti...
3    basically theres a family where a little boy j...
4    petter matteis love in the time of money is a ...
Name: review, dtype: object

In [4]:
df['review'][0]

'one of the other reviewers has mentioned that after watching just  oz episode youll be hooked they are right as this is exactly what happened with mebr br the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the wordbr br it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to manyaryans muslims gangstas latinos christians italians irish and moreso scuffles death stares dodgy dealings and shady agreements are never far awaybr br i would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare for

In [5]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [6]:
# 불용어 제거 및 동사 원형 변형
from nltk.corpus import stopwords
import nltk

result = []
wlem = nltk.WordNetLemmatizer()
cnt = 0

for sentence in df['review']:
    sentence = nltk.word_tokenize(sentence)
    result += [[wlem.lemmatize(i) for i in sentence if i not in stopwords.words('english')]]
    if cnt%100 == 0:print(cnt)
    cnt += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(result)
print(len(tokenizer.word_index))

164578


In [13]:
# 등장 빈도수가 3회 미만인 단어들의 분포 확인

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 164578
등장 빈도가 2번 이하인 희귀 단어의 수: 110696
단어 집합에서 희귀 단어의 비율: 67.26050869496531
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 2.1183526052462343


In [14]:
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 53883


In [15]:
tokenizer = Tokenizer(vocab_size) # 빈도수가 2번 이하인 단어를 제외
tokenizer.fit_on_texts(result)
x = tokenizer.texts_to_sequences(result) # 단어 -> 숫자(순서를 반영)

In [18]:
import numpy as np

df['sentiment'] = np.where(df['sentiment'] == 'positive',0,1) # 긍정은 0, 부정은 1로 변환
y = np.array(df['sentiment'])
df['sentiment'].values

array([0, 0, 0, ..., 1, 1, 1])

In [19]:
df['sentiment'].value_counts()

sentiment
0    25000
1    25000
Name: count, dtype: int64

In [20]:
drop_idx = [i for i in range(len(x)) if len(x[i])==0] # 길이가 0인 문장이 있는 인덱스
len(drop_idx)

0

In [21]:
cnt = 0 
for i in x:
    if len(i) <= 290:
        cnt += 1

print((cnt/len(x))*100) # 길이가 290이하인 리스트가 전체 리스트의 94.1%

94.17


In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x = pad_sequences(x,maxlen = 290) # max 길이를 290으로 조정

### LSTM(장기 단기 기억망)

- 단기 기억망인 RNN은 과거의 정보를 오랫동안 기억하지 못하는 단점이 있음
    - 이를 보완하기 위해 LSTM 탄생 


- 과거의 정보를 오래 기억하지 못하면
    - 정확도가 저하
    - 시퀀스 데이터(순서가 있는 데이터)를 처리하기가 어려움
    - 장기 기억이 가능하게 하기 위해 모델 복잡화

- Bidirectional 
    - 과거와 미래 정보를 모두 사용하여 현재 단어를 예측 ex) "나는"과 "먹는다"를 모두 보고 "사과"를 예측
    - 문맥을 더 잘 이해하여 해당 문장의 의미를 더 정확히 파악

In [29]:
from tensorflow.keras.layers import Embedding, Dense, LSTM,Bidirectional
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size, 128)) # (단어 수, 출력할 임베딩의 차원 수)
# -> 각 단어를 고유한 숫자 리스트로 나타냄, texts_to_sequences과 다르게 단어의 의미적 특징 반영

model.add(Bidirectional(LSTM(128))) # 장기 단기 기억망 (LSTM) 모델을 양방향으로 학습
# 128 :  LSTM 셀이 과거 정보를 저장하고 처리하는데 사용하는 벡터의 길이

model.add(Dense(1,activation='sigmoid'))

In [24]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

esc = EarlyStopping(patience=2,restore_best_weights=True)
mc = ModelCheckpoint('best_model2.h5',save_best_only=True)

In [26]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest = train_test_split(np.array(x),np.array(y),test_size=0.2,random_state=42)

In [27]:
xtrain, val_data, ytrain, val_test = train_test_split(xtrain,ytrain,test_size=0.2, random_state=42)
print(xtrain.shape,ytrain.shape)
print(val_data.shape,val_test.shape)

(32000, 290) (32000,)
(8000, 290) (8000,)


In [30]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',
              metrics='accuracy')
h = model.fit(xtrain,ytrain,epochs=15,callbacks=[esc,mc],
              batch_size=100,validation_data=(val_data,val_test))

Epoch 1/15
320/320 [==============================] - ETA: 0s - loss: 0.4901 - accuracy: 0.7486

c:\Users\amysm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


320/320 [==============================] - 1233s 4s/step - loss: 0.4901 - accuracy: 0.7486 - val_loss: 0.3257 - val_accuracy: 0.8695
Epoch 2/15
320/320 [==============================] - 1403s 4s/step - loss: 0.2808 - accuracy: 0.8909 - val_loss: 0.2810 - val_accuracy: 0.8821
Epoch 3/15
320/320 [==============================] - 1114s 3s/step - loss: 0.2154 - accuracy: 0.9206 - val_loss: 0.2911 - val_accuracy: 0.8838
Epoch 4/15
320/320 [==============================] - 1009s 3s/step - loss: 0.1756 - accuracy: 0.9375 - val_loss: 0.3039 - val_accuracy: 0.8717


In [31]:
evaluate_result = model.evaluate(xtest, ytest)
evaluate_result

313/313 [==============================] - 79s 249ms/step - loss: 0.2767 - accuracy: 0.8868


[0.27670201659202576, 0.8867999911308289]

In [44]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub('[^A-Za-z\s]', '', new_sentence).lower()
  new_sentence = nltk.word_tokenize(new_sentence)
  new_sentence = [wlem.lemmatize(word) for word in new_sentence if word not in stopwords.words('english')] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = 290) # 길이 맞춤
  score = float(model.predict(pad_new)) # 예측
  if(score < 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format((1 - score) * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format(score * 100))

In [51]:
new_sentence = "boring movie"
sentiment_predict(new_sentence)

1/1 [==============================] - 0s 86ms/step
90.02% 확률로 부정 리뷰입니다.



In [53]:
new_sentence = "perfect movie"
sentiment_predict(new_sentence)

1/1 [==============================] - 0s 138ms/step
76.33% 확률로 긍정 리뷰입니다.

